In [1]:
import polars as pl
from feature_store import Client
from feature_store.feature import FeatureKind
import sqlalchemy as sa

In [ ]:
# Data found here: https://www.kaggle.com/datasets/kapturovalexander/karpovs-sql-simulator

In [2]:
orders_df = pl.read_csv("data/orders.csv")

In [3]:
users_df = pl.read_csv("data/users.csv")

In [25]:
with pl.StringCache():
    users_df = pl.read_csv("data/users.csv").with_columns(
        pl.col("birth_date").str.strptime(pl.Date, "%d/%m/%y"), 
        pl.col("sex").cast(pl.Categorical),
        pl.lit("2022-01-01").str.strptime(pl.Date, "%Y-%m-%d").alias("date")
    )

    users = pl.concat([
        users_df,
        users_df.with_columns(
            pl.col("birth_date") + pl.duration(days=365),
            pl.col("sex"),
            pl.col("date") + pl.duration(days=365)
        )
    ])

In [4]:
meta = sa.MetaData()

In [6]:
engine = sa.create_engine("sqlite:///test.db")

In [7]:
meta.create_all(engine)

In [8]:
client = Client()

In [9]:
sex_feature = client.register_feature(
    feature_name="sex",
    kind=FeatureKind.sql,
    location="main.users",
    id_column="user_id",
    date_column="date",
    auth_key="local_sqlite"
)

In [27]:
client.upload_feature_data(sex_feature.name, users.to_pandas())

<Feature sex>

In [28]:
client.get_available_features()

[<Feature sex>]

In [29]:
f = client.get_feature("sex")

In [30]:
f.to_pandas()

,user_id,date,sex
0,10597,2022-01-01,male
1,4521,2022-01-01,female
2,5778,2022-01-01,male
3,8636,2022-01-01,female
4,15773,2022-01-01,male
...,...,...,...
40657,9797,2023-01-01,female
40658,1373,2023-01-01,male
40659,8747,2023-01-01,female
40660,338,2023-01-01,female
